### Importing Packages

In [1]:
import kaggle
import zipfile
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import warnings

### Downloading & Extracting Dataset

In [2]:
# # downloading the dataset
# !kaggle datasets download -d mohammadamireshraghi/blood-cell-cancer-all-4class
# # unziping the dataset 
# with zipfile.ZipFile('blood-cell-cancer-all-4class.zip','r') as zip_ref:
#     zip_ref.extractall('blood-cell-cancer-all-4class')

# print("Dataset downloaded and extracted successfully.")

### Set Up the Paths

In [3]:
base_dir="blood-cell-cancer-all-4class/Blood cell Cancer [ALL]"
train_dir= base_dir
test_dir= base_dir # Assuming the same directory structure for testing

### Loading & Preprocessing the data

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

#  set as training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (64,64),
    batch_size = 32,
    class_mode = "categorical",
    subset = "training"
)

# set as validation data
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (64,64),
    batch_size = 32,
    class_mode = "categorical",
    subset = "validation"
)

# set as testing data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (64,64),
    batch_size = 32,
    class_mode = "categorical"
)

Found 2595 images belonging to 4 classes.
Found 647 images belonging to 4 classes.
Found 3242 images belonging to 4 classes.


In [5]:
# checking dataset sizes
print("Training samples:", train_generator.samples)
print("Validation samples:", validation_generator.samples)

# calculating the steps per epoch
batch_size = 32
steps_per_epoch = max(1, train_generator.samples // batch_size) #  calculates the number of steps in each epoch.
validation_steps = max(1, validation_generator.samples // batch_size) # calculates the number of validation steps.

print(F"Steps Per Epoch: {steps_per_epoch}")
print(F"Validation Steps: {validation_steps}")

Training samples: 2595
Validation samples: 647
Steps Per Epoch: 81
Validation Steps: 20


### Build & train the CNN Model

In [6]:
warnings.filterwarnings("ignore") # ignoring warnings

In [7]:
# Building the MODEL
model = Sequential([
        Conv2D(32,(3,3), input_shape = (64,64,3), activation = "relu"),
        MaxPooling2D(pool_size = (2,2)),

        Conv2D(64, (3,3), activation = "relu"),
        MaxPooling2D(pool_size = (2,2)),

        Conv2D(128, (3,3), activation = "relu"),
        MaxPooling2D(pool_size = (2,2)),

        Flatten(),
        Dense(128, activation = "relu"),
        Dropout(0.5),
        Dense(4, activation = "softmax")
    ])

In [8]:
# compiling the model
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"]) 

In [9]:
# training the model
model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // train_generator.batch_size,
    epochs = 25,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples // validation_generator.batch_size
)

Epoch 1/25
28/81 ━━━━━━━━━━━━━━━━━━━━ 52s 985ms/step - accuracy: 0.2986 - loss: 1.3802

In [ ]:
# saving the model
model.save("blood_cancer_cnn_model.keras")

### Visualizing the Dataset

In [ ]:
sample_training_image,_ = next(train_generator)

def plotImages(images_arr):
    fig,axes = plt.subplots(1,5, figsize=(20,20))
    axes= axes.flatten()
    for img, ax in zip(images_arr,axes):
        ax.imshow(img)
        ax.axis("off")
    plt.tight_layout()
    plt.show()

plotImages(sample_training_image[:5])
    

# Testing the model

### Loading the model

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("blood_cancer_cnn_model.keras")

In [ ]:
images, labels = next(test_generator) # get a batch of images and labels

### make prediction

In [ ]:
predictions = model.predict(images)
 # decode the prediction
class_indices = test_generator.class_indices
class_labels = list(class_indices.keys())
predicted_classes = np.argmax(predictions, axis = 1)
predicted_labels =[class_labels[i] for i in predicted_classes]

### printing prediction

In [ ]:
for i, pred in enumerate(predicted_labels):
    print(F"Image {i+1}: predicted -{pred},\nActual - {class_labels[np.argmax(labels[i])]}")

In [ ]:
# define function to plot images with predictions and circles
import cv2
def plot_image_with_circle(image, true_label, predicted_label, radius=30, color=(255,0,0), thickness = 2):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    height, width, _ =image.shape
    center = (width // 2, height // 2)
    cv2.circle(image,center, radius, color, thickness)
    plt.imshow(image)
    plt.title(F"True: {true_label},\nPredicted: {predicted_label}")
    plt.axis("off")
    plt.show()

# visualize a few predictions with circles
for i in range (len(images)):
    plot_image_with_circle(images[i], class_labels[np.argmax(labels[i])], predicted_labels[i])

##Pickle

In [ ]:
import pickle as pkl